In [1]:
import pandas as pd
import re
import nltk as nlp
from sklearn.feature_extraction.text import CountVectorizer
nlp.download("wordnet")
nlp.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\oguzh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oguzh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
twitter = pd.read_csv("gender-classifier-DFE-791531.csv", encoding="latin1")
X = twitter.loc[:, ["gender", "text"]]
X.dropna(axis=0, inplace=True)
y = [1 if x == "male" else 0 for x in X.gender]
X.head()
len(X.text)

19953

In [3]:
len(X.text)

19953

In [4]:
def prepareData(data):
    series_to_array = data.values
    wnl = nlp.WordNetLemmatizer() 
    for i, v in enumerate(series_to_array):
        temp = re.sub("[^a-zA-z]", " ", v).lower()
        series_to_array[i] = " ".join([wnl.lemmatize(x) for x in nlp.word_tokenize(temp)])
    return data

X_final_text = prepareData(X["text"])
X_final_text

0        robbie e responds to critic after win against ...
1        it felt like they were my friend and i wa livi...
2        i absolutely adore when louis start the song i...
3        hi jordanspieth looking at the url do you use ...
4        watching neighbour on sky catching up with the...
                               ...                        
20045    lookupondeath fine and i ll drink tea too i lo...
20046    greg hardy you a good player and all but don t...
20047    you can miss people and still never want to se...
20048    bitemyapp i had noticed your tendency to pee o...
20049    i think for my apush creative project i m goin...
Name: text, Length: 19953, dtype: object

In [5]:
#extract word feature from data(convert to sparse matrix)
vectorizer = CountVectorizer(max_features=440, stop_words="english")
X_sparse = vectorizer.fit_transform(X_final_text)


In [6]:
print(vectorizer.get_feature_names())


['__', 'account', 'actually', 'ago', 'album', 'amas', 'amazing', 'american', 'amp', 'answer', 'app', 'apple', 'art', 'artist', 'artistoftheyear', 'ask', 'available', 'away', 'awesome', 'baby', 'bacon', 'bad', 'bc', 'beautiful', 'bed', 'believe', 'best', 'better', 'big', 'birthday', 'bit', 'bitch', 'black', 'blog', 'blue', 'body', 'bond', 'book', 'boy', 'break', 'bring', 'brother', 'build', 'building', 'business', 'buy', 'called', 'came', 'cancer', 'car', 'card', 'care', 'cat', 'cause', 'chance', 'change', 'channel', 'character', 'check', 'child', 'chill', 'city', 'class', 'click', 'club', 'come', 'coming', 'cool', 'costume', 'couldn', 'couple', 'cup', 'cut', 'cute', 'dad', 'damn', 'date', 'day', 'dead', 'deal', 'desk', 'did', 'didn', 'difference', 'different', 'digital', 'doe', 'doesn', 'dog', 'doing', 'don', 'dont', 'door', 'dream', 'drive', 'dude', 'early', 'eat', 'end', 'enjoy', 'episode', 'event', 'everydayiloveyou', 'excited', 'eye', 'face', 'facebook', 'fact', 'fall', 'family', '

In [7]:
#we have sparse matrix for these features
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split

#split train and test data
X_full = X_sparse.toarray()
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.15, random_state=23)

model = XGBClassifier(learning_rate=0.001, max_depth=8, n_jobs=-1)
model.fit(X_train, y_train)
print("model accuracy : {}".format(model.score(X_test, y_test)))
model.get_params()


model accuracy : 0.6912796525225526


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.001,
 'max_delta_step': 0,
 'max_depth': 8,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': -1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}